In [1]:
import pandas as pd
import spacy
from spacy import displacy
import os
import re
import string
import en_core_web_sm
from deep_translator import GoogleTranslator 
nlp = en_core_web_sm.load()

In [2]:
JobSkills = pd.read_excel("data/ResumeSkill_preprocessed.xlsx")
# lower characters and add spaces to the side to spot the complete words
# DE_JobSkills = (" " + JobSkills["Data_Engineer"].dropna().str.lower() + " ").to_list()
DE_JobSkills = JobSkills["Data_Engineer"].dropna().str.lower().to_list()
DE_JobSkills[0:10]

['api',
 'rest',
 'shell',
 'cli',
 'cronjobs',
 'python',
 'java',
 'scala',
 'go',
 'sql']

In [3]:
# bumeran_scraped_data_22-06-2022_1129.json
JSON_FILEPATH = "../../Bumeran/scraped_data/bumeran_scraped_data_27-05-2022_1638.json"
json_df = pd.read_json(JSON_FILEPATH, orient='records')

In [4]:
json_df.head()

,keyWords,title_role,description,date_published,schema,consulted_datetime,source,company_rate,salary,company,city
0,[desarrollador],Desarrollador de Negocios - CDMX,¡ÚNETE A NUESTRO EQUIPO!\nDESARROLLADOR DE NEG...,Publicado hace 7 días,Híbrido,"27/05/2022, 16:34",https://www.bumeran.com.mx/empleos/desarrollad...,NaN,"De $30,000 a $50,000 por mes",Confidencial,"Ciudad de México, Distrito Federal, Mexico"
1,[desarrollador],Desarrollador TI,Desarrollador TI\nIngeniería en sistemas compu...,Publicado hace más de 15 días,Presencial,"27/05/2022, 16:34",https://www.bumeran.com.mx/empleos/desarrollad...,NaN,No especificado,ENLACE LABORAL CONSULTORES RRHH S.C.,"Toluca de Lerdo, Estado de México, Mexico"
2,[desarrollador],Desarrollador Jr.,Desarrollador Jr.\nIngeniería en sistemas comp...,Publicado hace más de 15 días,Presencial,"27/05/2022, 16:35",https://www.bumeran.com.mx/empleos/desarrollad...,NaN,No especificado,ENLACE LABORAL CONSULTORES RRHH S.C.,"Toluca de Lerdo, Estado de México, Mexico"
3,[desarrollador],Desarrollador Java - Backend,Nosotros:\nSomos la empresa más reconocida en ...,Publicado hace más de 15 días,Presencial,"27/05/2022, 16:35",https://www.bumeran.com.mx/empleos/desarrollad...,NaN,No especificado,TRUPER SERVICIOS SA DE CV,"Miguel Hidalgo, Distrito Federal, Mexico"
4,[desarrollador],Desarrollador y Diseñador Web,Requisitos\nLicenciatura terminada\nConocimien...,Publicado hace más de 15 días,Presencial,"27/05/2022, 16:35",https://www.bumeran.com.mx/empleos/desarrollad...,NaN,No especificado,ASOCIACI?N MEXICANA DE DISTRIBUIDORES DE MAQUI...,"Miguel Hidalgo, Distrito Federal, Mexico"


In [5]:
rows_containing_salary = json_df.loc[json_df["description"].str.contains("Salario")]

In [6]:
json_df['blob_en'] = None

In [7]:
text = json_df['description'][0]

if len(text) >= 5000:
    # sentences = text.split(sep = '.')
    text = text.replace('\n', ' ')
    sentences = re.split(r'(?<=\D)\.(?=.)|(?<=\d)\.(?=\D)', text)

    # filter_object = filter(lambda x: x != "", sentences)
    filter_object = filter(lambda x: len(x) > 2, sentences)
    sentences = list(filter_object)

    filter_object = filter(lambda x: bool(re.search('[a-zA-Z]', x)), sentences) 
    sentences = list(filter_object)

    filter_object = filter(lambda x: (sum(c.isalpha() for c in x)) > 2, sentences)
    sentences = list(filter_object)

    translated = GoogleTranslator('es', 'en').translate_batch(sentences)
    translated_news = '. '.join(translated)
    json_df['blob_en'][0] = translated_news
else:
    translated_news = GoogleTranslator(source='es', target='en').translate(text)
    json_df['blob_en'][0] = translated_news

SSLError: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: /m?tl=en&sl=es&q=%C2%A1%C3%9ANETE+A+NUESTRO+EQUIPO%21%0ADESARROLLADOR+DE+NEGOCIOS%0AREQUISITOS%3A%0A%C2%B7+Experiencia+m%C3%ADnima+de+3+a+5+a%C3%B1os+en+ventas+de+productos+tangibles+e+intangibles+%28Preferentemente%29%0A%C2%B7+Escolaridad%3A+Licenciatura+en+Actuar%C3%ADa%2C+Finanzas%2C+Relaciones+P%C3%BAblicas%2C+Relaciones+Internacionales%2C+Administraci%C3%B3n%2C+Derecho%2C+MBA%2C+Ingenier%C3%ADas+afines+%28con+c%C3%A9dula+profesional%29%0A%C2%B7+Tolerancia+a+la+frustraci%C3%B3n%0A%C2%B7+Alto+nivel+de+energ%C3%ADa%0A%C2%B7+Enfoque+a+objetivos+y+metas%0A%C2%B7+Habilidad+financiera%0A%C2%B7+Gusto+por+las+ventas%0A%C2%B7+Poder+de+negociaci%C3%B3n%0A%C2%B7+Atenci%C3%B3n+a+clientes%0A%C2%B7+Facilidad+de+palabra%0AOFRECEMOS%3A%0A%C2%B7+Horario+Flexible+%28Lunes+a+Viernes%29%0A%C2%B7+Excelente+nivel+y+calidad+de+vida%0A%C2%B7+Altos+ingresos+econ%C3%B3micos%0A%C2%B7+Plan+de+carrera+creciente

In [124]:
SAMPLE_TEXT = json_df['blob_en'][0]
SAMPLE_TEXT

"We are a 100% Mexican company with more than 25 years of experience in connecting talent with companies.\n\n\nAnalytics Data Engineer in Monterrey, Guadalajara and AguasCalientes\nThe ideal candidate is a motivated individual who is excited about making an impact to a user base of over 20k that is ready to operate in a fast-paced environment and can clearly demonstrate a thirst for learning and mastering emerging technologies, processes and tools in the analyticsspace. This person should take initiative and shouldn't be an order taker. They should enjoy optimizing data models to industry standards.\n\n\nKey Requirements:\n\n1. Enterprise level SSAS / Tabular DAX Modeling experience\n2. Experience with SQL Querying\n3. Experienced with agile processes and methodologies\n4. Experience with Production Models & troubleshooting skills\n5. Ability to demonstrate understanding of business requirements & business concepts\n\nFollowing skills are not required but are a plus :\n\n6. Experience 

In [125]:
# Data Cleaning

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    # text = re.sub('\[.*?¿\]\%', ' ', text)
    text = re.sub(r'[^\w\s$]', ' ', text) 
    # text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    # text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('\n', ' ', text)
    return text
 
data_clean = clean_text_round1(SAMPLE_TEXT)
data_clean

'we are a 100  mexican company with more than 25 years of experience in connecting talent with companies    analytics data engineer in monterrey  guadalajara and aguascalientes the ideal candidate is a motivated individual who is excited about making an impact to a user base of over 20k that is ready to operate in a fast paced environment and can clearly demonstrate a thirst for learning and mastering emerging technologies  processes and tools in the analyticsspace  this person should take initiative and shouldn t be an order taker  they should enjoy optimizing data models to industry standards    key requirements   1  enterprise level ssas   tabular dax modeling experience 2  experience with sql querying 3  experienced with agile processes and methodologies 4  experience with production models   troubleshooting skills 5  ability to demonstrate understanding of business requirements   business concepts  following skills are not required but are a plus    6  experience working with a te

In [128]:
# { jobSkill:[x.span() for x in re.finditer(jobSkill,SAMPLE_TEXT.lower())]  for jobSkill in DE_JobSkills} 
JobSkillsLoc = { jobSkill:[x.span() for x in re.finditer(" "+ jobSkill + " ",data_clean)] for jobSkill in DE_JobSkills} 
JobSkillsLoc

{'api': [],
 'rest': [],
 'shell': [],
 'cli': [],
 'cronjobs': [],
 'python': [],
 'java': [],
 'scala': [],
 'go': [],
 'sql': [(700, 705)],
 'acid': [],
 'cap': [],
 'oltp': [],
 'olap': [],
 'mysql': [],
 'postgresql': [],
 'mariadb': [],
 'amazon aurora': [],
 'mongodb': [],
 'elasticsearch': [],
 'apache couchdb': [],
 'azure cosmosdb': [],
 'apache cassandra': [],
 'apache hbase': [],
 'google bigtable': [],
 'neo4j': [],
 'amazon neptune': [],
 'redis': [],
 'memcached': [],
 'amazon dynamodb': [],
 'snowflake': [],
 'presto': [],
 'apache hive': [],
 'apache impala': [],
 'amazon redshift': [],
 'azure synapse': [],
 'clickhouse': [],
 'apache hadoop': [],
 'hdfs': [],
 'mapreduce': [],
 'amazon mr': [],
 'google dataproc': [],
 'azure data lake': [],
 'apache pig': [],
 'apache arrow': [],
 'data build tool': [],
 'apache spark': [],
 'apache beam': [],
 'apache flink': [],
 'apache nifi': [],
 'apache kafka': [],
 'apache storm': [],
 'apache samza': [],
 'amazon kinesis': [

In [107]:
# nlp = spacy.load("es_core_news_sm")
nlp = en_core_web_sm.load()

# skill = "XPATH" 
# [skill[:],"Skill"]

# {"SKILL",""}
# Vales de despensa, "Beneficios"

In [111]:
doc = nlp(rows_containing_salary["description"].iloc[0])

In [112]:
for ent in doc.ents:
    print(ent.text,ent.label_)

100% PERCENT
de experiencia PERSON
con ORG
Analytics Data Engineer ORG
Monterrey GPE
Guadalajara GPE
20k DATE
SSAS / ORG
2 CARDINAL
SQL Querying ORG
3 CARDINAL
4 CARDINAL
Production Models ORG
5 CARDINAL
Git ORG
7 CARDINAL
Spark/Databricks Experience ORG
8 CARDINAL
English LANGUAGE
MXN  

 ORG
one CARDINAL
3 CARDINAL
Guadalajara GPE
Monterrey PERSON
English LANGUAGE
Cuajimalpa de Morelos PERSON
Distrito Federal ORG
apta para personas PERSON
El proceso de selección ORG
diseñado para PERSON
tu experiencia de PERSON
1 CARDINAL
2 CARDINAL
Revisar tu mail PERSON
3 CARDINAL
vemos que PERSON
tu perfil PERSON
estamos NORP
la GPE
